#### Лабораторная работа № 2. Разработка нейросетевых функций. Операция Convolution 3D
#### БВТ2003 Никитин Степан Романович

In [7]:
import torch
import numpy as np
from typing import Union

In [8]:
class Conv3D:
    def __init__(self, in_channels: int, out_channels: int, kernel_size: Union[int, tuple], stride: Union[int, tuple] = 1) -> None:
        """
        Конструктор класса Conv3D
        """
        self.in_channels = in_channels     # количество входных каналов (каналов входного тензора)
        self.out_channels = out_channels   # количество выходных каналов (каналов результата свертки)
        self.kernel_size = kernel_size     # размер ядра свертки (предполагается квадратное ядро)
        self.stride = stride               # шаг свертки

        # преобразование kernel_size в кортеж, если он целое число
        if isinstance(kernel_size, int):
            kernel_size = (kernel_size, kernel_size, kernel_size)

        # инициализация весов (ядер свертки)
        self.weights = np.random.randn(out_channels, in_channels, *kernel_size)
        self.bias = np.zeros((out_channels,))

    def forward(self, input_tensor: np.ndarray) -> np.ndarray:
        """
        Метод для выполнения операции свертки
        """
        in_channels, input_depth, input_height, input_width = input_tensor.shape
        kernel_size = self.kernel_size

        # преобразование kernel_size в кортеж, если он целое число
        if isinstance(kernel_size, int):
            kernel_size = (kernel_size, kernel_size, kernel_size)

        # проверка на корректные размеры входного тензора
        if input_depth < kernel_size[0] or input_height < kernel_size[1] or input_width < kernel_size[2]:
            raise ValueError("Неверный размер ядра для входного тензора")

        # вычисление размеров результата свертки
        if isinstance(self.stride, int):
            stride_d = stride_h = stride_w = self.stride
        elif isinstance(self.stride, tuple) and len(self.stride) == 3:
            stride_d, stride_h, stride_w = self.stride
        else:
            raise ValueError("Недопустимый формат шага. Должно быть целое число или кортеж из двух целых чисел")

        output_depth = (input_depth - kernel_size[0]) // stride_d + 1
        output_height = (input_height - kernel_size[1]) // stride_h + 1
        output_width = (input_width - kernel_size[2]) // stride_w + 1

        # инициализация результата свертки
        result = np.zeros((self.out_channels, output_depth, output_height, output_width))

        # выполнение свертки
        for d in range(0, input_depth - kernel_size[0] + 1, stride_d):
            for i in range(0, input_height - kernel_size[1] + 1, stride_h):
                for j in range(0, input_width - kernel_size[2] + 1, stride_w):
                    # окно свертки
                    window = input_tensor[:, d:d + kernel_size[0], i:i + kernel_size[1], j:j + kernel_size[2]]

                    # вычисление свертки и добавление bias
                    result[:, d // stride_d, i // stride_h, j // stride_w] = np.sum(window * self.weights, axis=(1, 2, 3, 4)) + self.bias

        return result

In [9]:
def run_3d_test(in_channels: int, out_channels: int, kernel_size: Union[int, tuple], stride: Union[int, tuple],
                padding: Union[int, tuple], dilation: int, groups: int, bias: bool, padding_mode: str) -> None:
    """
    Функция для выполнения теста трехмерной операции свертки
    """
    # создание экземпляра Conv3D
    conv3d_layer = Conv3D(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=stride)

    # создание трехмерного тензора для теста
    input_tensor_3d = np.random.randn(in_channels, 10, 10, 10)

    # выполнение трехмерной свертки
    result_3d = conv3d_layer.forward(input_tensor_3d)

    print(f"Тест с параметрами:")
    print(f"  - in_channels: {in_channels}")
    print(f"  - out_channels: {out_channels}")
    print(f"  - kernel_size: {kernel_size}")
    print(f"  - stride: {stride}")
    print(f"  - padding: {padding}")
    print(f"  - dilation: {dilation}")
    print(f"  - groups: {groups}")
    print(f"  - bias: {bias}")
    print(f"  - padding_mode: {padding_mode}")
    print(f"\nРезультат трехмерной свертки:\n\n {result_3d}")
    print(f"\n{'-'*75}\n")

In [10]:
def test_3d_params() -> None:
    """
    Функция для тестирования различных параметров трехмерной операции свертки
    """
    # параметры тестирования для первой группы
    run_3d_test(in_channels=8, out_channels=20, kernel_size=3, stride=1, padding=0, dilation=1, groups=2, bias=True, padding_mode='zeros')

    # параметры тестирования для второй группы
    run_3d_test(in_channels=4, out_channels=16, kernel_size=2, stride=1, padding=0, dilation=2, groups=1, bias=True, padding_mode='replicate')

    # параметры тестирования для третьей группы
    run_3d_test(in_channels=8, out_channels=4, kernel_size=2, stride=(2, 3, 2), padding=0, dilation=1, groups=2, bias=True, padding_mode='replicate')

    # параметры тестирования для четвертой группы
    run_3d_test(in_channels=5, out_channels=32, kernel_size=(3, 2, 1), stride=(4, 3, 2), padding=1, dilation=2, groups=1, bias=True, padding_mode='zeros')

In [11]:
# параметры тестирования для дополнительной группы
input_tensor_3d = np.random.randn(3, 5, 5, 5)
conv3d_layer = Conv3D(in_channels=3, out_channels=2, kernel_size=3, stride=(2, 2, 2))
output_tensor_3d = conv3d_layer.forward(input_tensor_3d)

print(f"Результат трехмерной свертки:\n {output_tensor_3d}")

Результат трехмерной свертки:
 [[[[-19.87194473  14.86470882]
   [ -9.17761468   2.12670617]]

  [[ -0.77851262 -12.53611   ]
   [  5.25326411  18.48580837]]]


 [[[  3.91983606   5.66679842]
   [ -2.01992921   4.95920916]]

  [[  7.67412691   0.62063987]
   [  9.42615173   4.49452861]]]]


In [12]:
test_3d_params()

Тест с параметрами:
  - in_channels: 8
  - out_channels: 20
  - kernel_size: 3
  - stride: 1
  - padding: 0
  - dilation: 1
  - groups: 2
  - bias: True
  - padding_mode: zeros

Результат трехмерной свертки:

 [[[[ 2.91685992e+01 -1.24443911e+01  8.20970642e+00 ...  8.98648541e+00
    -2.24712635e+00 -1.65898390e+01]
   [-4.81124355e+00 -1.36807627e+01 -1.06739866e+01 ... -6.71673652e+00
    -3.41163419e+00  2.99312188e+01]
   [ 1.24292912e+01  7.94567652e+00  2.87266663e+01 ...  7.05209569e+00
     1.65583766e+01 -2.40092654e+01]
   ...
   [-1.52320399e+01 -1.56950652e+01 -3.02407485e+01 ...  6.95280490e+00
    -1.54321093e+00 -2.57439472e+01]
   [ 4.08426436e+00  3.02155197e+00  1.23082045e+01 ...  7.34180732e+00
    -9.97145864e+00 -1.08808251e+00]
   [-6.25034591e+00  1.02075869e+01  1.35872747e+01 ...  1.80910420e+00
    -9.01337483e+00  3.36621360e+00]]

  [[ 1.46917496e+01 -2.08593794e+01  8.63479889e+00 ... -2.03308701e+01
    -1.96835239e+00 -1.01154622e+01]
   [-3.05846758e+0